In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
def metropolis_step(N, grid, J, beta=1.0):
    i, j = np.random.randint(0, N, 2)
    current_angle = grid[i, j]
    new_angle = 2*np.pi*np.random.rand()

    # Calculate energy change
    delta_E = 0
    delta_E -= J[i,j,0]*np.cos(new_angle-grid[(i + 1) % N, j])-np.cos(current_angle-grid[(i + 1) % N, j])
    delta_E -= J[i-1,j,0]*np.cos(new_angle-grid[(i - 1) % N, j])-np.cos(current_angle-grid[(i - 1) % N, j])
    delta_E -= J[i,j,1]*np.cos(new_angle-grid[i, (j + 1) % N])-np.cos(current_angle-grid[i, (j + 1) % N])
    delta_E -= J[i-1,j,1]*np.cos(new_angle-grid[i, (j - 1) % N])-np.cos(current_angle-grid[i, (j - 1) % N])

    # Metropolis criterion
    if delta_E < 0 or np.random.rand() < np.exp(-delta_E*beta):
        grid[i, j] = new_angle

    return grid


In [3]:
def plot_grid_orientations(grid, step, beta, filename_pattern):
    N = grid.shape[0]
    X, Y = np.meshgrid(range(N), range(N))
    U = np.cos(grid)
    V = np.sin(grid)

    plt.figure(figsize=(8, 8))
    plt.quiver(X, Y, U, V, pivot='middle')
    plt.title(f"Temperature {1/beta} Step {step}")
    plt.savefig(filename_pattern.format(beta, step))  # Save the plot as an image
    plt.close()  # Close the plot to free memory

In [4]:
# Modify your simulation loop to save images
image_folder = 'simulation_frames1'
os.makedirs(image_folder, exist_ok=True)
filename_pattern = os.path.join(image_folder, 'frame_{:02f}_{:04d}.png')

In [27]:
n = 5 #num of different J
N = 32  # Grid size
steps = 5000  # Number of simulation steps
m = 100 #num of thermalization
betas = [0.1, 10, 1000]
history = np.zeros((n, len(betas), steps+1, N, N), dtype='float32')
ods = np.zeros((n,len(betas), steps-m))
some_interval = 10



for k in range(n):
    #Initialize spin orientation.
    grid = 2*np.pi*np.random.rand(N, N)
    
    #Initialize spin-spin interaction coefficient.
    random_array = np.random.rand(N, N, 2)
    J = np.where(random_array < 0.5, -1, 1)

    for l in range(len(betas)):
        beta = betas[l]
        for h in range(steps):
            grid = metropolis_step(N, grid, J, beta)
            history[k, l, h, ...] = grid
            if h>m-1:
                ods[k,l,h-m] = np.mean(np.cos(grid))

            # Plotting at selected steps
            if h % some_interval == 0 and k == 0:  # 'some_interval' is the frequency of steps at which you want to plot
                plot_grid_orientations(grid, h, beta, filename_pattern)

    if k==0:
        break
odt = np.mean(ods, axis=2)**2
odt = np.mean(odt, axis=0)

In [28]:
# After simulation, create a video
video_name = 'simulation_video.mp4'
images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 24, (width, height))

for image in sorted(images):
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

In [ ]:
# Plotting the results
#plt.figure(figsize=(10, 6))
for i in range(3):
    plt.plot(range(len(ods[0,i])), ods[0,i])
plt.xlabel('Temperature')
plt.ylabel('Spin-Spin Correlation')
plt.title('Spin-Spin Correlation vs Temperature')
plt.grid(True)
plt.show()

In [ ]:
, marker='o'
序参数
可视化
不同的cooling rate  32-2048
1.直接到T
2.低温下cooling rate